In [1]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, Literal
import os
from notion_client import Client
from my_custom_tools.registry import custom_tool_registry
import shutil

load_dotenv(override=True)

# Fetch the Notion API key and set up client
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

# Initialize the Notion client
notion = Client(auth=notion_api_key)

In [2]:
papers = [{"title": "HoloFusion: Towards Photo-realistic 3D Generative Modeling", "summary": "Diffusion-based image generators can now produce high-quality and diverse\nsamples, but their success has yet to fully translate to 3D generation:\nexisting diffusion methods can either generate low-resolution but 3D consistent\noutputs, or detailed 2D views of 3D objects but with potential structural\ndefects and lacking view consistency or realism. We present HoloFusion, a\nmethod that combines the best of these approaches to produce high-fidelity,\nplausible, and diverse 3D samples while learning from a collection of\nmulti-view 2D images only. The method first generates coarse 3D samples using a\nvariant of the recently proposed HoloDiffusion generator. Then, it\nindependently renders and upsamples a large number of views of the coarse 3D\nmodel, super-resolves them to add detail, and distills those into a single,\nhigh-fidelity implicit 3D representation, which also ensures view consistency\nof the final renders. The super-resolution network is trained as an integral\npart of HoloFusion, end-to-end, and the final distillation uses a new sampling\nscheme to capture the space of super-resolved signals. We compare our method\nagainst existing baselines, including DreamFusion, Get3D, EG3D, and\nHoloDiffusion, and achieve, to the best of our knowledge, the most realistic\nresults on the challenging CO3Dv2 dataset.", "link": "https://arxiv.org/pdf/2308.14244v1.pdf"}, {"title": "HoloDiffusion: Training a 3D Diffusion Model using 2D Images", "summary": "Diffusion models have emerged as the best approach for generative modeling of\n2D images. Part of their success is due to the possibility of training them on\nmillions if not billions of images with a stable learning objective. However,\nextending these models to 3D remains difficult for two reasons. First, finding\na large quantity of 3D training data is much more complex than for 2D images.\nSecond, while it is conceptually trivial to extend the models to operate on 3D\nrather than 2D grids, the associated cubic growth in memory and compute\ncomplexity makes this infeasible. We address the first challenge by introducing\na new diffusion setup that can be trained, end-to-end, with only posed 2D\nimages for supervision; and the second challenge by proposing an image\nformation model that decouples model memory from spatial memory. We evaluate\nour method on real-world data, using the CO3D dataset which has not been used\nto train 3D generative models before. We show that our diffusion models are\nscalable, train robustly, and are competitive in terms of sample quality and\nfidelity to existing approaches for 3D generative modeling.", "link": "https://arxiv.org/pdf/2303.16509v2.pdf"}]

tool = PaperSummary0Tool()
result = tool.run(None, papers)

NameError: name 'PaperSummary0Tool' is not defined

In [3]:
my_config = Config.from_default()
complete_tool_registry = PortiaToolRegistry(my_config) + custom_tool_registry

portia = Portia(config = my_config,
                tools = complete_tool_registry,
                execution_hooks=CLIExecutionHooks(),)

In [5]:
constraints = []
# topic = input("What topic are you interested in covering today?")
# number_of_papers = int(input("How many papers do you want to download?"))
topic = "Holodiffusion"
video_preference = False
rec_reading_preference = False
quiz_preference = True

# Define the path to your papers folder
papers_folder = "papers"

# Ensure the papers folder exists
if not os.path.exists(papers_folder):
    os.makedirs(papers_folder)
else:
    # If it exists, clear its contents
    for filename in os.listdir(papers_folder):
        file_path = os.path.join(papers_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # remove file or symlink
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # remove directory
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')


task = (
            lambda : f"""You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of {topic} using the ArXivTool. 
                      - Use PSTool to create and populate the Page Summary subpage.
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - From the text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.
                      - Then use the TopicSelectorTool on these topics. 
                      - Then use the Notion Tool to create Notion pages for these topics.

                      - {video_preference * "Use the YouTubeTool to find videos on each topic."}
                      - {rec_reading_preference * "Use the RecReadTool to find resources on each topic."}
                      - {quiz_preference * "Use the QuizTool to create quizzes on each topic."}
                      
                      
                        Take into account these constraints: {constraints}
                      """

        )

                     #- Run the PaperSummary0 to populate the Page Summary subpage.
# task = (
#             lambda : f"""You are a research assistant running these tasks: 
#                       - Find 1 paper on the topic of {topic} using the ArXivTool. 
#                       - Run the PSTool to create and populate the Page Summary subpage.
         
#                         Take into account these constraints: {constraints}
#                       """

#         )




# Iterate on the plan with the user until they are happy with it
with execution_context(end_user_id="learning_enthusiast",):
    plan = portia.plan(task())
    print("\nHere are the steps in the generated plan:")
    [print(step.model_dump_json(indent=2)) for step in plan.steps]
    ready_to_proceed = False
    while not ready_to_proceed:
        user_input = input("Are you happy with the plan? (y/n):\n")
        if user_input == "y":
            ready_to_proceed = True
        else:
            user_input = input("Any additional guidance for the planner?:\n")
            constraints.append(user_input)
            plan = portia.plan(task())
            print("\nHere are the updated steps in the plan:")
            [print(step.model_dump_json(indent=2)) for step in plan.steps]

    # Execute the plan
    print("\nThe plan will now be executed. Please wait...")
    run = portia.run_plan(plan)
    
    if run.state != PlanRunState.COMPLETE:
        raise Exception(
            f"Plan run failed with state {run.state}. Check logs for details."
        )

2025-04-13 04:17:56.505 | INFO | portia.portia:plan:197 - Running planning_agent for query - You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of Holodiffusion using the ArXivTool. 
                      - Use PSTool to create and populate the Page Summary subpage.
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - From the text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.
                      - Then use the TopicSelectorTool on these topics. 
                      - Then use the Notion Tool to create